In [38]:
zip_file_path = (r'E:\Capstone DS\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET')

data = (r'E:\Capstone DS\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET')

In [17]:
%pip install google-colab

from google.colab import drive
import zipfile
import os

os.makedirs(data, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data)

extracted_files = os.listdir(data)
print(extracted_files)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google-colab (from versions: none)
ERROR: No matching distribution found for google-colab


ModuleNotFoundError: No module named 'google.colab'

In [21]:
train = (r'E:\Capstone DS\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET\train')
test = (r'E:\Capstone DS\FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET')

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np

In [23]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

Found 32398 images belonging to 3 classes.
Found 42898 images belonging to 2 classes.


In [24]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [39]:
initial_learning_rate = 0.001

def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [44]:
layers.InputLayer(shape=(None,)),
layers.Dense(512, activation="relu"),
layers.Dense(256, activation="relu"),
layers.GlobalAveragePooling1D(),
layers.Dense(3, activation="softmax")
epochs = 1
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=test_generator,
                    callbacks=[lr_scheduler])


evaluation_result = model.evaluate(test_generator)
print(f'Test Loss: {evaluation_result[0]}, Test Accuracy: {evaluation_result[1]}')

1013/1013 ━━━━━━━━━━━━━━━━━━━━ 0s 847ms/step - accuracy: 0.9653 - loss: 0.0978

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 3)

In [ ]:
save_dir = 'content'
os.makedirs(save_dir, exist_ok=True)
model.save(os.path.join(save_dir, 'fire_cnn.h5'))

In [40]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np


user_image_path1 = '/Fire (36).jpg'
user_image1 = image.load_img(user_image_path1, target_size=(224, 224))
user_image_array1 = image.img_to_array(user_image1)
user_image_array1 = np.expand_dims(user_image_array1, axis=0)
user_image_array1 = preprocess_input(user_image_array1)

predictions1 = model.predict(user_image_array1)

predicted_class1 = np.argmax(predictions1)
predicted_label1 = list(train_generator.class_indices.keys())[predicted_class1]

plt.imshow(user_image1)
plt.title(f'Predicted Class: {predicted_label1}')
plt.show()


user_image_path2 = '/smoke1.jpeg'
user_image2 = image.load_img(user_image_path2, target_size=(224, 224))
user_image_array2 = image.img_to_array(user_image2)
user_image_array2 = np.expand_dims(user_image_array2, axis=0)
user_image_array2 = preprocess_input(user_image_array2)


predictions2 = model.predict(user_image_array2)

predicted_class2 = np.argmax(predictions2)
predicted_label2 = list(train_generator.class_indices.keys())[predicted_class2]


plt.imshow(user_image2)
plt.title(f'Predicted Class: {predicted_label2}')
plt.show()

user_image_path3 = '/images.jpg'
user_image3 = image.load_img(user_image_path3, target_size=(224, 224))
user_image_array3 = image.img_to_array(user_image3)
user_image_array3 = np.expand_dims(user_image_array3, axis=0)
user_image_array3 = preprocess_input(user_image_array3)


predictions3 = model.predict(user_image_array3)


predicted_class3 = np.argmax(predictions3)
predicted_label3 = list(train_generator.class_indices.keys())[predicted_class3]
plt.imshow(user_image3)
plt.title(f'Predicted Class: {predicted_label3}')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

def load_images_from_folder(folder):
    images = []
    labels = []
    label_map = {'smoke': 0, 'fire': 1, 'non_fire': 2}
    for label, label_idx in label_map.items():
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            images.append(img_path)
            labels.append(label_idx)
    return images, labels


train_images, train_labels = load_images_from_folder(train)
test_images, test_labels = load_images_from_folder(test)

pred_labels = np.argmax(model.predict(test_images), axis=1)

conf_matrix = confusion_matrix(test_labels, pred_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['smoke', 'fire', 'non_fire'], yticklabels=['smoke', 'fire', 'non_fire'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

accuracy = accuracy_score(test_labels, pred_labels)
print(f'Accuracy Score: {accuracy:.4f}')